- 2020.04.28

----

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.model_selection import train_test_split

import glob
import os
import multiprocessing

import pickle
import time
import random

---

In [2]:
def chunkIt(seq, num):
    if len(seq) % int(num) == 0:
        avg = int(len(seq) / int(num))
    else:
        avg = int(len(seq) / int(num)) + 1 # 이렇게 해야하네!
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg

    return out       

---

# data load

## # Train data

In [3]:
with open('../train_within_0_35_balanced/X_tain.pkl', 'rb') as f:
    X_train = pickle.load( f )
    
with open('../train_within_0_35_balanced/y_train.pkl', 'rb') as f:
    y_train = pickle.load( f )

In [4]:
print(X_train.shape, y_train.shape)

(330519, 512) (330519,)


---

## ## Test data

In [5]:
with open('../test_data(common)/TEST_within_035/test_pair_v2_035.pkl', 'rb') as f:
    X_test_dict = pickle.load( f )
    
with open('../test_data(common)/TEST_within_035/test_label_035.pkl', 'rb') as f:
    y_test_dict = pickle.load( f )

In [6]:
len(X_test_dict)

100

In [7]:
len(y_test_dict)

100

In [8]:
# test data 100개 array로

X_test = np.array([])
y_test = np.array([])

for i,key in enumerate(X_test_dict.keys()):
    if i == 0:
        X_test = X_test_dict[key]
        y_test = y_test_dict[key]
    else:
        X_test = np.append( X_test, X_test_dict[key], axis=0 )
        y_test = np.append( y_test, y_test_dict[key], axis=0 )

In [9]:
print(X_test.shape, y_test.shape)

(2183, 512) (2183,)


---
---

## ## Training

In [10]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout


Using TensorFlow backend.


In [12]:
# Generate dummy data

model = Sequential()
model.add(Dense(256, input_dim=512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train,
          epochs=50,
          batch_size=512, validation_data = (X_test, y_test))


Epoch 1/50
646/646 [==============================] - 3s 5ms/step - loss: 0.6451 - accuracy: 0.6332 - val_loss: 0.8771 - val_accuracy: 0.3225
Epoch 2/50
646/646 [==============================] - 3s 5ms/step - loss: 0.6251 - accuracy: 0.6387 - val_loss: 1.1211 - val_accuracy: 0.2790
Epoch 3/50
646/646 [==============================] - 3s 5ms/step - loss: 0.6057 - accuracy: 0.6486 - val_loss: 1.1066 - val_accuracy: 0.3426
Epoch 4/50
646/646 [==============================] - 3s 5ms/step - loss: 0.5861 - accuracy: 0.6620 - val_loss: 1.3986 - val_accuracy: 0.3871
Epoch 5/50
646/646 [==============================] - 3s 5ms/step - loss: 0.5681 - accuracy: 0.6757 - val_loss: 1.1344 - val_accuracy: 0.4544
Epoch 6/50
646/646 [==============================] - 3s 5ms/step - loss: 0.5508 - accuracy: 0.6886 - val_loss: 1.7789 - val_accuracy: 0.4146
Epoch 7/50
646/646 [==============================] - 3s 5ms/step - loss: 0.5356 - accuracy: 0.6995 - val_loss: 1.3363 - val_accuracy: 0.4530
Epoch 

In [13]:
# test_set positive 비율
np.sum(y_test)/len(y_test)

0.16720109940448924

In [14]:
y_pred = model.predict( X_test )

In [15]:
y_pred

array([[0.09593717],
       [0.04652655],
       [0.49291888],
       ...,
       [0.9994784 ],
       [0.99985576],
       [0.9999013 ]], dtype=float32)

In [17]:
np.sum( 1*( y_pred.ravel() < 0.5 ) )/len(y_pred)

0.3820430600091617

In [34]:
( 1*( y_pred.ravel() < 0.5 ) ).shape

(7368000,)

In [35]:
( 1*( y_pred.ravel() < 0.5 ) ).reshape(100,-1).shape

(100, 73680)

In [22]:
predicted

array([1, 1, 1, ..., 0, 0, 0])

In [23]:
from sklearn import metrics

precision_100 = []
recall_100 = []

grp_pred_w_key = {}
grp_label_w_key = {}

test_label = y_test
predicted = ( 1*( y_pred.ravel() > 0.5 ) )

#start test key
start_test_key = list(y_test_dict.keys())[0]
#init start index
start_index = 0

for test_key in y_test_dict.keys(): 
    print(test_key)
    if test_key == start_test_key:
        start_index = 0
    
    test_obj_grp_size = len( y_test_dict[test_key] )
    grp_pred = predicted[start_index : start_index + test_obj_grp_size]
    grp_label = test_label[start_index : start_index +test_obj_grp_size]
    # 저장
    grp_pred_w_key[test_key] = grp_pred
    grp_label_w_key[test_key] = grp_label
    ####
    
    print(start_index, test_obj_grp_size)
    
    start_index += test_obj_grp_size
    
    print('label_size : {}'.format(np.sum( grp_label )), 
          'prediction_size : {}'.format(np.sum( grp_pred )) )
    
    # calculate precision / recall
    precision_ = metrics.precision_score( grp_label , grp_pred )
    recall_ = metrics.recall_score( grp_label , grp_pred )
    
    # print
    print( precision_ )
    print( recall_ )
    print('\n')
    
    # append precision / recall
    precision_100.append( precision_ )
    recall_100.append( recall_ )

40648
0 10
label_size : 2 prediction_size : 2
0.5
0.5


66052
10 17
label_size : 2 prediction_size : 17
0.11764705882352941
1.0


37783
27 5
label_size : 2 prediction_size : 2
1.0
1.0


59761
32 5
label_size : 2 prediction_size : 5
0.4
1.0


16639
37 4
label_size : 4 prediction_size : 4
1.0
1.0


17958
41 5
label_size : 5 prediction_size : 5
1.0
1.0


10918
46 18
label_size : 5 prediction_size : 18
0.2777777777777778
1.0


26542
64 4
label_size : 4 prediction_size : 4
1.0
1.0


71651
68 5
label_size : 2 prediction_size : 5
0.4
1.0


45696
73 2
label_size : 1 prediction_size : 2
0.5
1.0


4253
75 12
label_size : 8 prediction_size : 12
0.6666666666666666
1.0


63461
87 96
label_size : 2 prediction_size : 84
0.023809523809523808
1.0


572
183 17
label_size : 14 prediction_size : 17
0.8235294117647058
1.0


66460
200 1
label_size : 0 prediction_size : 0
0.0
0.0


70828
201 24
label_size : 1 prediction_size : 3
0.3333333333333333
1.0


34741
225 170
label_size : 4 prediction_size : 94
0.042

In [24]:
np.mean(precision_100)

0.6214972510989937

In [25]:
np.mean(recall_100)

0.9325

---